In [1]:
# 必要なライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import os
import re
import glob
import shutil
import gc
import pyarrow as pa
from pathlib import Path

# matplotlibの日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# グラフの表示方法調整
sns.set(rc={'figure.figsize':(12,5)});
plt.figure(figsize=(12,5));

# 乱数の種
random_seed = 123

<Figure size 864x360 with 0 Axes>

In [2]:
# タイマー
# https://github.com/nyk510/vivid/blob/master/vivid/utils.py
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

## 機械学習とは
機械学習とは、ざっと言ってしまうとあるデータ X を入力として対応する予測値 y を取り出すような対応関係を作成することです。

例：タイタニック号で、乗客が生きるか死ぬかを予測する問題だと X は乗客の年齢, 性別, 船室のグレード… など乗客に紐づく情報のことを指します。通常、この情報のことを特徴量とよびます。

特徴量 X と 予測値 y が用意できれば学習用データ (X - y の関係がわかっているデータ) を元にして X をいれて y になるようにモデルを調整する。この調整の段階を学習とよびます。学習には様々なアルゴリズムがあるが、X, y を用意しなくてはならない部分は基本的に変わらないです。

# 今回のコンペに関して

コンペ自体のデータサイズが大きいのでデータハンドリングが僕たちに取っては肝になりそうです

> データの読み込みに関して
* 形式を変えたデータセットをpandasで読み込む
  * [`feather`](https://www.kaggle.com/datasets/munumbutt/amexfeather) or [`Parquet`](https://www.kaggle.com/datasets/odins0n/amex-parquet)
* [`pyspark`](https://www.kaggle.com/code/rakkaalhazimi/export-large-dataset-to-spark) or [`dask`](https://docs.dask.org/en/latest/dataframe.html) で読み込む

> サイズが大きい場合の対処方法は以下のkaggle notebookを参考にしてください

[How to Work with BIG Datasets on 16G RAM (+Dask)](https://www.kaggle.com/code/yuliagm/how-to-work-with-big-datasets-on-16g-ram-dask)

以下がnotebookの内容概要
* TIP 1 - 使用していない変数を [`delete`](https://www.sejuku.net/blog/74447) するか & gc.collect()` で[ガベージコレクション](https://techacademy.jp/magazine/19437)(不要になったメモリ領域を開放して再利用する機能)をする
* TIP 2 - データセット内の各カラムのデータタイプを予め定義しておく
  * eg: 本来 float64 だったものを -> float32 と定義してサイズを圧縮する
* TIP 3 - 読み込むデータセットを選択する (including generating your own subsamples)
  * 読み込む行数を選ぶ
  * (`skiprows`) で[読み飛ばす行数を指定する](https://bit.ly/3O90Ze7)
  * 読み飛ばすリストを作成して、読み飛ばす（以下例）
    * ```
      skiplines = np.random.choice(np.arange(1, lines), size=lines-1-1000000, replace=False)
      skiplines=np.sort(skiplines)
      train = pd.read_csv('../input/train.csv', skiprows=skiplines, dtype=dtypes)
      ```
* TIP 4 - バッチ処理をする
  * ひとまとまりのデータに対して、一連の処理を連続で実行する処理方式のこと。大きなデータに関しても、設定したデータ数(チャンク)ごとに処理をする
* TIP 5 - 特定のカラムのみ `import` する
  * 450,000行 × 150カラム より 100万行 × 2カラム の方がメモリ消費が少ないことは容易に想像できる
* TIP 6 - groupby などの処理をするときも一部カラムの一部データのみにするˆ
* TIP 7 - `Dask` を使用する。
  * [DaskについてのQiita記事](https://qiita.com/simonritchie/items/e174f243bc03fb25462e)

## コンペの概要
* コンペ名：[American Express - Default Prediction](https://www.kaggle.com/competitions/amex-default-prediction)

* 目的：毎月の顧客プロファイルから、顧客がクレジットカードの残高分を将来返済しない確率を予測すること
  * ターゲットのバイナリ変数は、最新のクレジットカード明細書から18ヶ月間のパフォーマンスウィンドウを観察することによって計算され、顧客が最新の明細書の日付から120日以内に支払額を支払わない場合、デフォルトとみなされる。

* 評価方法(Evaluation)
  * このコンペではクレカのデフォルト率を予測する。サブミットはちょっと特殊な評価方法で評価される。以下の通り:
    * ```
      M = 0.5*(G+D)  (*G = 正規化ジニ係数, D = デフォルト率 4% )
      ```
      機械学習における `正規化ジニ係数` は経済学などで使用される ジニ係数とは違うので注意。以下記事参照
      * [機械学習のモデル評価、説明可能性のための指標　その１。ジニ係数とAUC](https://qiita.com/Derek/items/4ded249f7a75f8da176c)
      * [DataRobot](https://docs.datarobot.com/ja/docs/modeling/reference/model-detail/opt-metric.html#gini-coefficient)
      * [GINI and AUC relationship](https://stats.stackexchange.com/questions/342329/gini-and-auc-relationship)
      * [Why use Normalized Gini Score instead of AUC as evaluation?](https://stats.stackexchange.com/questions/306287/why-use-normalized-gini-score-instead-of-auc-as-evaluation)

* データについて
  * データセットには各顧客の特徴が各明細書日付ごとに集約されたものが含まれている。特徴は匿名化され、正規化されており、以下のカテゴリに分類されている（カラムの prefix を見ると分かる）:
    * D_*: Delinquency variables
    * S_*: Spend variables
    * P_*: Payment variables
    * B_*: Balance variables
    * R_*: Risk variables
  * 以下のカラムはカテゴリ変数である:
    * B_30, B_38, D_63, D_64, D_66, D_68, D_114, D_116, D_117, D_120, D_126

# データ管理環境整備

≒ディレクトリ整備

In [6]:
# # input_dir（input directory） を作る
current_note_path = os.path.dirname(os.path.abspath('__file__'))
INPUT_DIR = os.path.join(current_note_path, "data")

# INPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(INPUT_DIR):
    os.mkdir(INPUT_DIR)

# output_dir(output directory) を作る
OUTPUT_DIR = os.path.join(current_note_path, 'outputs')

# OUTPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [4]:
# 一度やったらコメントアウトする

# # csvファイルを `data` ディレクトリ（=フォルダー） に移動させる
# unique_dir_names = []
# for f in Path(f'{current_note_path}').rglob('*.csv'):
#     unique_dir_names.append(f)

# for file in list(set(unique_dir_names)):
#     print(f'moved file: {file}')
#     shutil.move(f'{file}', f'{INPUT_DIR}')

moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/test_data.csv
moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/train_labels.csv
moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/train_data.csv
moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/sample_submission.csv


In [15]:
# csv(or any file)を簡単に読み込めるようにする関数
# csv を読み取る関数を設定したあげると、pathや拡張子を書かずに読み込めるので入力が楽になる

# 通常のcsvファイルを読み取るときは以下
def read_csv(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.csv')
    print(f'Load: {path}')
    return pd.read_csv(path, **kwrgs)

# 今回はparquet形式を読み込むので別途用意
def read_parquet(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.parquet')
    print(f'Load: {path}')
    return pd.read_parquet(path, **kwrgs)

# データ読み込み

前記の通り、データサイズが大きいため、pandas などを安易に使ってもローカルではメモリが足りないので、ここでは2つの方法を取る
* parquet 形式のデータを読み込む (*本来は自分でcsv -> parquet形式に変える必要があるが、心優しい人がparquet形式にしてくれているのでそれを参照する。)
  * [該当データダウンロードページ](https://www.kaggle.com/datasets/odins0n/amex-parquet)
  * [Load Parquet Files with Low Memory](https://www.kaggle.com/code/odins0n/load-parquet-files-with-low-memory)
  * [parquetデータを使用したEDA](https://www.kaggle.com/code/odins0n/amex-default-prediction-detailed-eda)
* a

In [14]:
#test_df = read_parquet('test_data')
train_df = read_parquet('train_data')
train_labels = read_csv('train_labels')
sample_sub = read_csv('sample_submission')

Load: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/data/test_data.parquet


KeyboardInterrupt: 

In [ ]:
# ガベージコレクション
gc.collect()

In [29]:
train_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 190 entries, customer_ID to D_145
dtypes: object(4), float64(185), int64(1)

In [ ]:
# 以下のカラムは頻出で、毎回入力するのはめんどくさいので、ポップアップされるように定義します

# 各テーブルの概説

# EDA (データ確認)

## 欠損値・異常値確認

# 前処理 + 特徴量エンジニアリング

## カテゴリ変数の変換

## データマージ

# 学習・予測・サブミットファイル作成

## モデル構築

## モデル評価

## サブミットファイル作成